In [21]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import random
tf.reset_default_graph()

# Fashion MNIST

손글씨 말고 옷을 분류하자

옷 사진을 28 * 28 의 픽셀 데이터로 변환해서 나타냄


0 T-shirt/top

1 Trouser

2 Pullover

3 Dress

4 Coat

5 Sandal

6 Shirt

7 Sneaker

8 Bag

9 Ankle boot 


### load data

train set과 test 셋을 불러오자

데이터 출처 : https://www.kaggle.com/zalando-research/fashionmnist

In [3]:
df_train=pd.read_csv('fashion-mnist_train.csv')
df_test =pd.read_csv('fashion-mnist_test.csv')

df_train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


불러온 데이터의 모양을 확인해보자

In [4]:
print('train shape:',df_train.shape)
print('test shape:',df_test.shape)

train shape: (60000, 785)
test shape: (10000, 785)


데이터는 784개의 픽셀데이터+ 1개의 라벨로 이뤄져 있다.

train 데이터는 6만개, test 데이터는 만개 있다.

### set data : train set

학습을 위한 train set을 정해주자

In [22]:
train_set=df_train.as_matrix(None)


x_train=train_set[:,1:]
y_train=train_set[:,[0]]

print('x_train shape:',x_train.shape)
print('y_train shape:',y_train.shape)

x_train shape: (60000, 784)
y_train shape: (60000, 1)


한번에 6만개의 데이터를 학습시키기에는 너무 많은 양이다.

따라서 batch_size = 100 으로해서 100개씩 나눠서 학습시키자

배치를 넣어줄려면 single element로 넣어줘야한다.

tf.train.slice_input_producer 으로 잘라서 얻은 single element (queue op)을 tf.train.batch에 넣으면 된다.

In [23]:
x_bt = tf.train.input_producer(tf.constant(np.asarray(x_train)))
y_bt = tf.train.input_producer(tf.constant(np.asarray(y_train)))

x_bt=x_bt.dequeue()
y_bt=y_bt.dequeue()

print(x_bt)
print(y_bt)

Tensor("FIFOQueueV2_Dequeue:0", shape=(784,), dtype=int64)
Tensor("FIFOQueueV2_Dequeue_1:0", shape=(1,), dtype=int64)


In [24]:
batch_size=100
train_x_batch, train_y_batch = tf.train.batch([x_bt,y_bt] , batch_size=100)

print(train_x_batch.shape)
print(train_y_batch.shape)

(100, 784)
(100, 1)


### set data : test set

test 세트를 설정하자

In [25]:
test_set=df_test.as_matrix(columns=None)

x_test=test_set[:,1:]
y_test=test_set[:,[0]]

print('x_test shape:',x_test.shape)
print('y_test shape:',y_test.shape)

x_test shape: (10000, 784)
y_test shape: (10000, 1)


test세트는 따로 배치 해줄 필요 없다.

## 모델 만들기

### 첫번째 레이어

X:우리가 입력할 사진은 (60000, 784)와 (10000,784)이므로 28X28로 바꿔줘야 한다.

Y:원핫 인코딩이 적용되므로 출력은 10개가 되야한다.

In [26]:
X= tf.placeholder(tf.float32, [None,784])
X_img=tf.reshape(X, [-1,28,28,1])


Y = tf.placeholder(tf.int32, [None, 1])  

Y_one_hot = tf.one_hot(Y, 10)  # one hot
print("one_hot", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, 10])
print("reshape", Y_one_hot)

one_hot Tensor("one_hot:0", shape=(?, 1, 10), dtype=float32)
reshape Tensor("Reshape_1:0", shape=(?, 10), dtype=float32)


이제 첫번째 필터를 만들어주자

첫번째 필터는 3X3 크기로 하고 색은 1개, 32개의 필터를 사용하자 -> (3,3,1,32)

In [27]:
W1=tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))
L1= tf.nn.conv2d(X_img,W1,strides=[1,1,1,1],padding='SAME')

print('conv2d:',L1.shape)

conv2d: (?, 28, 28, 32)


stride는 1X1로 하고 padding까지 적용해주면 모양은 (?,28,28,32)일 것이다.

통과시키고 렐루 함수도 적용시키자

In [28]:
L1=tf.nn.relu(L1)

print('Relu:',L1.shape)

Relu: (?, 28, 28, 32)


Max-pooling도 통과시켜주면 첫번째 레이어가 끝난다.

In [29]:
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

print(L1.shape)

(?, 14, 14, 32)


스트라이드 2X2 이므로 크기는 (?, 28, 28, 32)에서 절반으로 줄어든 (?, 14, 14, 32)가 될것이다

### 두번째 레이어

두번째 레이어를 통과시키자.

이때 들어오는 값의 크기는 Max-pooling을 통과하고 나온 (?, 14, 14, 32)가 될것이다

두번째 필터도 3X3 크기에 64개로 만들자. 단 이번에는 32개의 그림이 들어으므로 ([3,3,32,64])로 해줘야한다.

이하 과정은 첫번째 레이어와 같으므로 생략한다.

In [30]:
W2=tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))

L2=tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')

L2=tf.nn.relu(L2)

L2= tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

print(L2.shape)

(?, 7, 7, 64)


두번째 레이어에 들어올땐 (?, 14, 14, 32) 였지만 나갈땐 (?, 7, 7, 64)이 됬다.

### FC layer

지금 두번째 레이어까지 통과한 그림은 입체적인 모양이므로 이것을 한줄로 나열해야 한다.

In [31]:
L2= tf.reshape(L2,[-1,7*7*64])

print(L2.shape)

(?, 3136)


받은 7X7X64를 10개(숫자들의 원핫인코딩: 0~9)으로 출력한다.

In [32]:
W3=tf.get_variable("W3",shape=[7*7*64,10],initializer=tf.contrib.layers.xavier_initializer())
b=tf.Variable(tf.random_normal([10]))

hypothesis = tf.matmul(L2,W3)+b

### loss/cost function
모델을 만들었으므로 loss 함수와 학습시킬 준비를 하자

In [33]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y_one_hot))
optimizer =tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)

### Accuracy
추후 추가

### Graph Initialize


In [34]:
sess= tf.Session()
sess.run(tf.global_variables_initializer())

### Start populating

In [35]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

### Training

에포치는 15번하자

In [36]:
learning_rate = 0.001
training_epochs = 15
batch_size=100
total_batch= int(len(x_train)/batch_size)

학습 가즈ㅏㅏㅏㅏ

In [37]:
for epoch in range(training_epochs):
    avg_cost=0
    for i in range(total_batch):
        x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
        c, _ = sess.run([cost, optimizer],feed_dict={X:x_batch, Y:y_batch })
        avg_cost += c/ total_batch
    print('Epoch:','%04d'%(epoch+1),'cost=','{:.9f}'.format(avg_cost))

coord.request_stop()
coord.join(threads)    
print('Learning finish')    

Epoch: 0001 cost= 87.735053628
Epoch: 0002 cost= 2.308885290
Epoch: 0003 cost= 2.309404034
Epoch: 0004 cost= 2.309935186
Epoch: 0005 cost= 2.309535871
Epoch: 0006 cost= 2.309541063
Epoch: 0007 cost= 2.310039814
Epoch: 0008 cost= 2.309964692
Epoch: 0009 cost= 2.310167116
Epoch: 0010 cost= 2.308746410
Epoch: 0011 cost= 2.309949851
Epoch: 0012 cost= 2.309539884
Epoch: 0013 cost= 2.309675223
Epoch: 0014 cost= 2.309732673
Epoch: 0015 cost= 2.309723699
Learning finish
